In [59]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/forest-fires-regression/forestfires.csv


# Imports

In [60]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, mean_squared_error, make_scorer
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
import matplotlib.pyplot as plt


# Forest Fire data representation

In [61]:
df = pd.read_csv("/kaggle/input/forest-fires-regression/forestfires.csv")
df.describe()

,X,Y,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area
count,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000
mean,4.669246,4.299807,90.644681,110.872340,547.940039,9.021663,18.889168,44.288201,4.017602,0.021663,12.847292
std,2.313778,1.229900,5.520111,64.046482,248.066192,4.559477,5.806625,16.317469,1.791653,0.295959,63.655818
min,1.000000,2.000000,18.700000,1.100000,7.900000,0.000000,2.200000,15.000000,0.400000,0.000000,0.000000
25%,3.000000,4.000000,90.200000,68.600000,437.700000,6.500000,15.500000,33.000000,2.700000,0.000000,0.000000
50%,4.000000,4.000000,91.600000,108.300000,664.200000,8.400000,19.300000,42.000000,4.000000,0.000000,0.520000
75%,7.000000,5.000000,92.900000,142.400000,713.900000,10.800000,22.800000,53.000000,4.900000,0.000000,6.570000
max,9.000000,9.000000,96.200000,291.300000,860.600000,56.100000,33.300000,100.000000,9.400000,6.400000,1090.840000


# Implementing diff models and algorithms

## Helper functions & Data Preprocessing

In [62]:
def evaluate_model(model_name, y_test, y_pred):
    mad = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    print(f"{model_name} Performance:")
    print(f"Mean Absolute Deviation (MAD): {mad}")
    print(f"Root Mean Squared Error (RMSE): {rmse}\n")



In [63]:
# Correct feature names based on the dataset
numerical_features = ['temp', 'RH', 'wind', 'rain']  # Corrected based on dataset
categorical_features = []  # Leave empty if 'month' and 'day' are unavailable

# Preprocess the data
def preprocess_data(df, target, categorical_features, numerical_features):
    # Log-transform the target variable
    y = np.log1p(df[target])  
    
    # Remove the target column from the features
    X = df.drop(columns=[target])
    
    # Define transformers
    preprocess = ColumnTransformer(
        transformers=[
            ('num', StandardScaler(), numerical_features),
            ('cat', OneHotEncoder(), categorical_features)
        ]
    )
    
    # Apply transformations
    X_transformed = preprocess.fit_transform(X)
    
    # Split the dataset into train and test sets
    return train_test_split(X_transformed, y, test_size=0.2, random_state=42)



### Neural network

In [64]:
def neural_network(X_train, X_test, y_train, y_test):
    model = MLPRegressor(hidden_layer_sizes=(10,), max_iter=1000, solver='adam',learning_rate_init=0.001,early_stopping=True, random_state=42)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    evaluate_model("Neural Network", y_test, y_pred)


### Optimimzing MLP

In [65]:
# Cross-Validation for MLP
def cross_validate_mlp(X, y):
    model = MLPRegressor(hidden_layer_sizes=(10,), max_iter=500, solver='adam', learning_rate_init=0.001, random_state=42)
    scorer = make_scorer(mean_squared_error, squared=False)
    scores = cross_val_score(model, X, y, cv=5, scoring=scorer)
    print("Cross-validated RMSE scores:", scores)
    print("Mean RMSE:", scores.mean())

# Hyperparameter Tuning for MLP
def tune_mlp(X_train, y_train):
    param_grid = {
        'hidden_layer_sizes': [(10,), (50,), (100,)],
        'learning_rate_init': [0.001, 0.01, 0.1],
        'solver': ['adam', 'sgd'],
        'max_iter': [500, 1000]
    }
    
    model = MLPRegressor(random_state=42, early_stopping=True)
    grid_search = GridSearchCV(model, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
    grid_search.fit(X_train, y_train)
    
    best_model = grid_search.best_estimator_
    print("Best Parameters:", grid_search.best_params_)
    print("Best RMSE:", np.sqrt(-grid_search.best_score_))
    return best_model


In [66]:
# Define target and features
target_column = 'area'
numerical_features = ['temp', 'RH', 'wind', 'rain']
categorical_features = []  # Add categorical features if needed

# Preprocess data
X_train, X_test, y_train, y_test = preprocess_data(df, target_column, categorical_features, numerical_features)

# Cross-validate MLP
print("Cross-Validation Results for MLP:")
cross_validate_mlp(X_train, y_train)

# Hyperparameter tuning for MLP
print("\nHyperparameter Tuning for MLP:")
best_mlp = tune_mlp(X_train, y_train)

# Evaluate best MLP on the test set
y_pred = best_mlp.predict(X_test)
evaluate_model("Tuned MLP Regressor", y_test, y_pred)

Cross-Validation Results for MLP:


/opt/conda/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


Cross-validated RMSE scores: [1.41556921 1.26431999 1.70147067 1.54545318 1.26838491]
Mean RMSE: 1.4390395931493056

Hyperparameter Tuning for MLP:


/opt/conda/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


Best Parameters: {'hidden_layer_sizes': (50,), 'learning_rate_init': 0.001, 'max_iter': 500, 'solver': 'sgd'}
Best RMSE: 1.3859581187939796
Tuned MLP Regressor Performance:
Mean Absolute Deviation (MAD): 1.1573045980401029
Root Mean Squared Error (RMSE): 1.453821275015725



## SVM


In [67]:
def support_vector_machine(X_train, X_test, y_train, y_test):
    model = SVR(kernel='rbf', C=3, epsilon=0.1, gamma='scale')
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    evaluate_model("Support Vector Machine", y_test, y_pred)


### Optimizing SVM

In [68]:
# Cross-validate SVM
def cross_validate_svm(X, y):
    model = SVR(kernel='rbf', C=3, epsilon=0.1, gamma='scale')
    scorer = make_scorer(mean_squared_error, squared=False)  # Use RMSE
    scores = cross_val_score(model, X, y, cv=5, scoring=scorer)
    
    # Print cross-validation scores
    print("Cross-Validation Results for SVM:")
    for i, score in enumerate(scores, 1):
        print(f"Fold {i}: RMSE = {score:.4f}")
    print(f"Mean RMSE: {np.mean(scores):.4f}")
    print(f"Standard Deviation of RMSE: {np.std(scores):.4f}\n")

# Hyperparameter tuning for SVM
def tune_svm(X_train, y_train):
    param_grid = {
        'C': [0.1, 1, 10, 100],
        'epsilon': [0.01, 0.1, 0.5, 1],
        'gamma': ['scale', 'auto', 0.01, 0.1]
    }
    model = SVR(kernel='rbf')
    grid_search = GridSearchCV(model, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
    grid_search.fit(X_train, y_train)
    
    # Extract best parameters and results
    best_model = grid_search.best_estimator_
    best_params = grid_search.best_params_
    best_score = np.sqrt(-grid_search.best_score_)  # Convert negative MSE to RMSE
    
    print("Hyperparameter Tuning Results for SVM:")
    print(f"Best Parameters: {best_params}")
    print(f"Best RMSE: {best_score:.4f}\n")
    
    return best_model

### Evaluation

In [69]:
# Evaluate SVM
print("Cross-Validation Results for SVM:")
cross_validate_svm(X_train, y_train)

print("\nHyperparameter Tuning for SVM:")
best_svm = tune_svm(X_train, y_train)

# Evaluate best SVM on the test set
y_pred = best_svm.predict(X_test)
evaluate_model("Tuned Support Vector Machine", y_test, y_pred)

Cross-Validation Results for SVM:
Cross-Validation Results for SVM:
Fold 1: RMSE = 1.4865
Fold 2: RMSE = 1.4091
Fold 3: RMSE = 1.6665
Fold 4: RMSE = 1.6731
Fold 5: RMSE = 1.2988
Mean RMSE: 1.5068
Standard Deviation of RMSE: 0.1458


Hyperparameter Tuning for SVM:
Hyperparameter Tuning Results for SVM:
Best Parameters: {'C': 100, 'epsilon': 1, 'gamma': 0.01}
Best RMSE: 1.3699

Tuned Support Vector Machine Performance:
Mean Absolute Deviation (MAD): 1.1675611358607194
Root Mean Squared Error (RMSE): 1.4712687197338465

